### R tutorial   

* Book:   
https://r4ds.had.co.nz/  
* Tidy R references   
https://tidyr.tidyverse.org/reference/
* Non tidy data   
https://simplystatistics.org/2016/02/17/non-tidy-data/

## Chapter 12: Tidy data
* observations: per row
* variables: per column
* values: per value

### Procedures of tidy data
Advantage: Tidy data allows easy "mutation" (computation based on exisitng variables per observation) of the dataset.  
1. Put each dataset in a tibble.
2. Put each variable in a column.

### More aggregation functions
* ```count(df, vars = NULL, wt_var = NULL)```:   
    - ```var```: id variables to count unique values of    
    - ```wt_var```: optional variable to weight by - if this is non-NULL, count will sum up the value of this variable for each combination of id variables  


### Pivoting
* ```pivot_longer(data, cols, names_to = "name", names_prefix = NULL, values_to = "value")```:   
* ```pivot_wider(data, cols, names_from = "name", names_prefix = "", values_from = "value")```:   

```pivot_longer()``` and ```pivote_wider()``` are not 100% reverse operations of each other. 
when values are turned into columns by pivot_wider() it becomes characters, 
when try to change columns back to values using ```pivot_longer()``` you need  ```names_ptype = list(year = double())```

### Separate
```separate()``` pulls apart one column into multiple columns, by splitting wherever a separator character. 
```R
separate(data, col, into, sep = "[^[:alnum:]]+", remove = TRUE,
  convert = FALSE, extra = "warn", fill = "warn", ...)   
```
```sep``` The default value is a regular expression that matches any sequence of non-alphanumeric values. if it is a number ```sep = 100```, it means to split from index position of 100.   
```convert = T``` convert data into better types accordingly to the situation.     
### Unite
```unite()``` is the inverse of ```separate()```: it combines multiple columns into a single column.
```R
unite(data, col, ..., sep = "_", remove = TRUE, na.rm = FALSE)

```

### Relational data functions:
https://dplyr.tidyverse.org/reference/join.html   
Groups are ignored for the purpose of joining, but the result preserves the grouping of x.

* ```join(x, y, by, copy = F, suffix = c('x', 'y'))```: x and y are dataframes to join. ```by``` is basically "SQL ```on```". Pass a vector or ```NULL``` to ```by```.
* Variations: ```inner_join(), left_join(),right_join(), full_join(), semi_join(), nest_join(), anti_join()```
* ```anti_join()```:return all rows from x where there are not matching values in y, keeping just columns from x.
* ```nest_join()```:  
    - Return all rows and all columns from x. Adds a list column of tibbles. Each tibble contains all the rows from y that match that row of x. When there is no match, the list column is a 0-row tibble with the same column names and types as y. nest_join() is the most fundamental join since you can recreate the other joins from it. An inner_join() is a nest_join() plus an tidyr::unnest(), and left_join() is a nest_join() plus an unnest(.drop = FALSE). A semi_join() is a nest_join() plus a filter() where you check that every element of data has at least one row, and an anti_join() is a nest_join() plus a filter() where you check every element has zero rows.


### Missing Values

* Missing values happens when pivoting. Use ```values_drop_na = T``` to drop ```NA```
* Use ```complete()``` to make it explicit. ```complete()``` takes a set of columns, and finds all unique combinations. It then ensures the original dataset contains all those values, filling in explicit ```NAs``` where necessary.

### Case Study tips:
* ```str_replace()``` vs ```sub()``` vs ```gsub()```
 - variable order is different. 
 - ```str_replace()``` and ```sub()``` replace the 1st occurance, whereas ```gsub()```replaces all.

In [1]:
suppressPackageStartupMessages(library(tidyverse))


In [4]:
# tidy data example:
table1 %>% head(3)

country,year,cases,population
<chr>,<int>,<int>,<int>
Afghanistan,1999,745,19987071
Afghanistan,2000,2666,20595360
Brazil,1999,37737,172006362


In [9]:
# compute cases per year
table1 %>%
    count(year, wt=cases)%>% 
    head(3)

year,n
<int>,<int>
1999,250740
2000,296920


### Pivoting Examples:

In [12]:
# untidy data: variables appears as column names
table4a%>% 
    head(3)

table4b %>% 
    head(3)

country,1999,2000
<chr>,<int>,<int>
Afghanistan,745,2666
Brazil,37737,80488
China,212258,213766


country,1999,2000
<chr>,<int>,<int>
Afghanistan,19987071,20595360
Brazil,172006362,174504898
China,1272915272,1280428583


In [11]:
# format to longer 
table4a %>% 
  pivot_longer(c(`1999`, `2000`), names_to = "year", values_to = "cases")

country,year,cases
<chr>,<chr>,<int>
Afghanistan,1999,745
Afghanistan,2000,2666
Brazil,1999,37737
Brazil,2000,80488
China,1999,212258
China,2000,213766


In [17]:
# untidy data: variables appears as column names
table2%>% 
    head(4)


country,year,type,count
<chr>,<int>,<chr>,<int>
Afghanistan,1999,cases,745
Afghanistan,1999,population,19987071
Afghanistan,2000,cases,2666
Afghanistan,2000,population,20595360


In [19]:
# format to wider
table2%>%
    pivot_wider( names_from = 'type', values_from = 'count' ) %>%
    head(4)

country,year,cases,population
<chr>,<int>,<int>,<int>
Afghanistan,1999,745,19987071
Afghanistan,2000,2666,20595360
Brazil,1999,37737,172006362
Brazil,2000,80488,174504898


In [28]:
# when values are turned into columns by pivot_wider() it becomes characters, 
# when try to change columns back to values you need  names_ptype = list(year = double())

(stocks <- tibble(
  year   = c(2015, 2015, 2016, 2016),
  half  = c(   1,    2,     1,    2),
  return = c(1.88, 0.59, 0.92, 0.17)
) )
stocks %>% 
  pivot_wider(names_from = year, values_from = return) %>% 
  pivot_longer(`2015`:`2016`, names_to = "year", values_to = "return", names_ptype = list(year = double()) )%>%
table4a %>% 
  pivot_longer(c('1999', '2000'), names_to = "year", values_to = "cases", names_ptypes = list(year = double()))  head()

year,half,return
<dbl>,<dbl>,<dbl>
2015,1,1.88
2015,2,0.59
2016,1,0.92
2016,2,0.17


half,year,return
<dbl>,<dbl>,<dbl>
1,2015,1.88
1,2016,0.92
2,2015,0.59
2,2016,0.17


In [33]:
# # this will not run at all
# table4a %>% 
#   pivot_longer(c(1999, 2000), names_to = "year", values_to = "cases")

# this will have character type for years
table4a %>% 
  pivot_longer(c('1999', '2000'), names_to = "year", values_to = "cases", names_ptypes = list(year = double()))
# this will have double type for years
table4a %>% 
  pivot_longer(c('1999', '2000'), names_to = "year", values_to = "cases", names_ptypes = list(year = double()))

country,year,cases
<chr>,<dbl>,<int>
Afghanistan,1999,745
Afghanistan,2000,2666
Brazil,1999,37737
Brazil,2000,80488
China,1999,212258
China,2000,213766


country,year,cases
<chr>,<dbl>,<int>
Afghanistan,1999,745
Afghanistan,2000,2666
Brazil,1999,37737
Brazil,2000,80488
China,1999,212258
China,2000,213766


#### Exercise 3: What would happen if you widen this table? Why? How could you add a new column to uniquely identify each value?

In [4]:
people <- tribble(
  ~name,             ~names,  ~values,
  #-----------------|--------|------
  "Phillip Woods",   "age",       45,
  "Phillip Woods",   "height",   186,
  "Phillip Woods",   "age",       50,
    
  "Jessica Cordero", "age",       37,
  "Jessica Cordero", "height",   156
)

In [47]:
# attempt 1
# people$index = 1:nrow(people)
people %>%
    pivot_wider(names_from = 'names', values_from = 'values')

Warning message:
“Values in `values` are not uniquely identified; output will contain list-cols.
* Use `values_fn = list(values = list)` to suppress this warning.
* Use `values_fn = list(values = length)` to identify where the duplicates arise
* Use `values_fn = list(values = summary_fun)` to summarise duplicates”


name,age,height
<chr>,<list>,<list>
Phillip Woods,"45, 50",186
Jessica Cordero,37,156


In [54]:
# attempt 2
people_1 <- people %>%
    mutate(index = 1:nrow(people))
people_1 %>%
    pivot_wider(names_from = 'names', values_from = 'values')

name,index,age,height
<chr>,<int>,<dbl>,<dbl>
Phillip Woods,1,45,NA
Phillip Woods,2,NA,186
Phillip Woods,3,50,NA
Jessica Cordero,4,37,NA
Jessica Cordero,5,NA,156


In [13]:
# attempt 3: this works because of luck
people %>%
  group_by(name) %>%
  mutate(row = row_number()) %>%
  filter(row <= 2) %>%
  select(-row) %>%
  pivot_wider(names_from = names, values_from = values) 

name,age,height
<chr>,<dbl>,<dbl>
Phillip Woods,45,186
Jessica Cordero,37,156


In [19]:
# attempt 4: this works rigorously
people %>%
  group_by(name, names) %>% # here is the difference
  mutate(row = row_number()) %>%
  filter(row <= 1) %>%
  select(-row) %>%
  pivot_wider(names_from = names, values_from = values) 

name,age,height
<chr>,<dbl>,<dbl>
Phillip Woods,45,186
Jessica Cordero,37,156


#### Exercise 4: Tidy this.

In [20]:
preg <- tribble(
  ~pregnant, ~male, ~female,
  "yes",     NA,    10,
  "no",      20,    12
)

In [23]:
preg %>%
    pivot_longer('male':'female', names_to = 'gender', values_to = 'counts')

pregnant,gender,counts
<chr>,<chr>,<dbl>
yes,male,NA
yes,female,10
no,male,20
no,female,12


### Separate
```separate()``` pulls apart one column into multiple columns, by splitting wherever a separator character. 

table3  the ```rate``` column is combination of ```cases``` and ```population``` 

In [26]:
table3

,country,year,rate
,<chr>,<int>,<chr>
1,Afghanistan,1999,745/19987071
2,Afghanistan,2000,2666/20595360
3,Brazil,1999,37737/172006362
4,Brazil,2000,80488/174504898
5,China,1999,212258/1272915272
6,China,2000,213766/1280428583


In [27]:
table3 %>% 
  separate(rate, into = c("cases", "population"), convert = TRUE)

country,year,cases,population
<chr>,<int>,<int>,<int>
Afghanistan,1999,745,19987071
Afghanistan,2000,2666,20595360
Brazil,1999,37737,172006362
Brazil,2000,80488,174504898
China,1999,212258,1272915272
China,2000,213766,1280428583


In [28]:
table3 %>% 
  separate(year, into = c("century", "year"), sep = 2)  

country,century,year,rate
<chr>,<chr>,<chr>,<chr>
Afghanistan,19,99,745/19987071
Afghanistan,20,00,2666/20595360
Brazil,19,99,37737/172006362
Brazil,20,00,80488/174504898
China,19,99,212258/1272915272
China,20,00,213766/1280428583


### Unite

In [29]:
table5

,country,century,year,rate
,<chr>,<chr>,<chr>,<chr>
1,Afghanistan,19,99,745/19987071
2,Afghanistan,20,00,2666/20595360
3,Brazil,19,99,37737/172006362
4,Brazil,20,00,80488/174504898
5,China,19,99,212258/1272915272
6,China,20,00,213766/1280428583


In [30]:
# vanila unite() sep = '_'
table5 %>% 
  unite(new, century, year)

country,new,rate
<chr>,<chr>,<chr>
Afghanistan,19_99,745/19987071
Afghanistan,20_00,2666/20595360
Brazil,19_99,37737/172006362
Brazil,20_00,80488/174504898
China,19_99,212258/1272915272
China,20_00,213766/1280428583


In [31]:
# specify sep
table5 %>% 
  unite(new, century, year, sep = '')

country,new,rate
<chr>,<chr>,<chr>
Afghanistan,1999,745/19987071
Afghanistan,2000,2666/20595360
Brazil,1999,37737/172006362
Brazil,2000,80488/174504898
China,1999,212258/1272915272
China,2000,213766/1280428583


In [34]:
tibble(x = c("a,b,c", "d,e,f,g", "h,i,j")) %>% 
  separate(x, c("one", "two", "three"), extra = 'merge') # default: extra = 'warn' drop extra with a warning

one,two,three
<chr>,<chr>,<chr>
a,b,c
d,e,"f,g"
h,i,j


In [39]:
tibble(x = c("a,b,c", "d,e", "f,g,i")) %>% 
  separate(x, c("one", "two", "three"), fill = 'left') 
# default fill = 'warn', fill with NA from right with warning. 'right' from right

one,two,three
<chr>,<chr>,<chr>
a,b,c
NA,d,e
f,g,i


### Joins

In [15]:
# join
tidy4a <- table4a %>% 
  pivot_longer(c(`1999`, `2000`), names_to = "year", values_to = "cases")
tidy4b <- table4b %>% 
  pivot_longer(c(`1999`, `2000`), names_to = "year", values_to = "population")
(table_4ab_joined <- left_join(tidy4a, tidy4b))

Joining, by = c("country", "year")



country,year,cases,population
<chr>,<chr>,<int>,<int>
Afghanistan,1999,745,19987071
Afghanistan,2000,2666,20595360
Brazil,1999,37737,172006362
Brazil,2000,80488,174504898
China,1999,212258,1272915272
China,2000,213766,1280428583


### Missing Values

In [40]:
(stocks <- tibble(
  year   = c(2015, 2015, 2015, 2015, 2016, 2016, 2016),
  qtr    = c(   1,    2,    3,    4,    2,    3,    4),
  return = c(1.88, 0.59, 0.35,   NA, 0.92, 0.17, 2.66)
))

year,qtr,return
<dbl>,<dbl>,<dbl>
2015,1,1.88
2015,2,0.59
2015,3,0.35
2015,4,NA
2016,2,0.92
2016,3,0.17
2016,4,2.66


In [46]:
stocks %>%
    complete(year, qtr)

year,qtr,return
<dbl>,<dbl>,<dbl>
2015,1,1.88
2015,2,0.59
2015,3,0.35
2015,4,NA
2016,1,NA
2016,2,0.92
2016,3,0.17
2016,4,2.66


In [44]:
# explicit by default when pivoting
stocks %>% 
  pivot_wider(names_from = year, values_from = return)

qtr,2015,2016
<dbl>,<dbl>,<dbl>
1,1.88,NA
2,0.59,0.92
3,0.35,0.17
4,NA,2.66


In [45]:
# implicity be dropping NA
stocks %>% 
  pivot_wider(names_from = year, values_from = return) %>% 
  pivot_longer(
    cols = c(`2015`, `2016`), 
    names_to = "year", 
    values_to = "return", 
    values_drop_na = TRUE
  )

qtr,year,return
<dbl>,<chr>,<dbl>
1,2015,1.88
2,2015,0.59
2,2016,0.92
3,2015,0.35
3,2016,0.17
4,2016,2.66


### Case Study
who dataset 

In [51]:
nrow(who)
ncol(who)

[1] 7240

[1] 60

In [48]:
who %>% head()

country,iso2,iso3,year,new_sp_m014,new_sp_m1524,new_sp_m2534,new_sp_m3544,new_sp_m4554,new_sp_m5564,⋯,newrel_m4554,newrel_m5564,newrel_m65,newrel_f014,newrel_f1524,newrel_f2534,newrel_f3544,newrel_f4554,newrel_f5564,newrel_f65
<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Afghanistan,AF,AFG,1980,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Afghanistan,AF,AFG,1981,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Afghanistan,AF,AFG,1982,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Afghanistan,AF,AFG,1983,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Afghanistan,AF,AFG,1984,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Afghanistan,AF,AFG,1985,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [70]:
(who1 <- who %>%
    pivot_longer('new_sp_m014':'newrel_f65', 
                 names_to = 'key', 
                 values_to = 'cases',
                 values_drop_na = TRUE)) %>%
    head()


country,iso2,iso3,year,key,cases
<chr>,<chr>,<chr>,<int>,<chr>,<int>
Afghanistan,AF,AFG,1997,new_sp_m014,0
Afghanistan,AF,AFG,1997,new_sp_m1524,10
Afghanistan,AF,AFG,1997,new_sp_m2534,6
Afghanistan,AF,AFG,1997,new_sp_m3544,3
Afghanistan,AF,AFG,1997,new_sp_m4554,5
Afghanistan,AF,AFG,1997,new_sp_m5564,2


In [60]:
who1 %>% 
    count(key) %>% # count is more like sum() in python
    head()

key,n
<chr>,<int>
new_ep_f014,1032
new_ep_f1524,1021
new_ep_f2534,1021
new_ep_f3544,1021
new_ep_f4554,1017
new_ep_f5564,1017


In [71]:
# str_replace() vs sub() vs gsub()
# variable order is different. str_replace() and sub() replace the 1st occurance, whereas gsub() replaces all.

(who2 <- who1 %>% 
  mutate(names_from = stringr::str_replace(key, "newrel", "new_rel"))) %>%
    tail(3)

(who2 <- who1 %>% 
  mutate(names_from = gsub("newrel", "new_rel", key))) %>%
    tail(3)

country,iso2,iso3,year,key,cases,names_from
<chr>,<chr>,<chr>,<int>,<chr>,<int>,<chr>
Zimbabwe,ZW,ZWE,2013,newrel_f4554,1453,new_rel_f4554
Zimbabwe,ZW,ZWE,2013,newrel_f5564,811,new_rel_f5564
Zimbabwe,ZW,ZWE,2013,newrel_f65,725,new_rel_f65


country,iso2,iso3,year,key,cases,names_from
<chr>,<chr>,<chr>,<int>,<chr>,<int>,<chr>
Zimbabwe,ZW,ZWE,2013,newrel_f4554,1453,new_rel_f4554
Zimbabwe,ZW,ZWE,2013,newrel_f5564,811,new_rel_f5564
Zimbabwe,ZW,ZWE,2013,newrel_f65,725,new_rel_f65


In [73]:
(who3 <- who2 %>% 
  separate(key, c("new", "type", "sexage"), sep = "_")) %>%
    head()

Warning message:
“Expected 3 pieces. Missing pieces filled with `NA` in 2580 rows [243, 244, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 903, 904, 905, 906, ...].”


country,iso2,iso3,year,new,type,sexage,cases,names_from
<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>
Afghanistan,AF,AFG,1997,new,sp,m014,0,new_sp_m014
Afghanistan,AF,AFG,1997,new,sp,m1524,10,new_sp_m1524
Afghanistan,AF,AFG,1997,new,sp,m2534,6,new_sp_m2534
Afghanistan,AF,AFG,1997,new,sp,m3544,3,new_sp_m3544
Afghanistan,AF,AFG,1997,new,sp,m4554,5,new_sp_m4554
Afghanistan,AF,AFG,1997,new,sp,m5564,2,new_sp_m5564


In [79]:
(who4 <- who3 %>% 
  separate(sexage, c("sex", "age"), sep = 1) %>%
  separate(age, c("age_start","age_end"), sep = -2) %>%
  unite("age_group", "age_start":"age_end", sep = '-') )%>%
    head()

country,iso2,iso3,year,new,type,sex,age_group,cases,names_from
<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>
Afghanistan,AF,AFG,1997,new,sp,m,0-14,0,new_sp_m014
Afghanistan,AF,AFG,1997,new,sp,m,15-24,10,new_sp_m1524
Afghanistan,AF,AFG,1997,new,sp,m,25-34,6,new_sp_m2534
Afghanistan,AF,AFG,1997,new,sp,m,35-44,3,new_sp_m3544
Afghanistan,AF,AFG,1997,new,sp,m,45-54,5,new_sp_m4554
Afghanistan,AF,AFG,1997,new,sp,m,55-64,2,new_sp_m5564
